In [47]:
%config IPCompleter.greedy=True

import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam

import numpy as np
import gym
from scipy import signal

In [49]:
class Actor(nn.Module):
    def __init__(self, obs_dimensions, act_dimensions):
        super().__init__()
        self.actor = nn.Sequential(
            nn.Linear(obs_dimensions, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, act_dimensions),
            nn.Tanh()
        )
        
    def get_distribution(self, obs):
        return Categorical(logits=self.actor(obs))
    
    def forward(self, obs, act=None):
        # Produce action distributions for given observations, and 
        # optionally compute the log likelihood of given actions under
        # those distributions
        pi = self.get_distribution(obs)
        logp_a = None
        if act is not None:
            logp_a = pi.log_prob(act)
        return pi, logp_a
        
class Critic(nn.Module):
    def __init__(self, obs_dimensions):
        super().__init__()
        self.critic = nn.Sequential(
            nn.Linear(obs_dimensions, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Tanh()
        )
    
    def forward(self, obs):
        return torch.squeeze(self.critic(obs), -1)

class ActorCritic(nn.Module):
    def __init__(self, obs_space, act_space):
        super().__init__()
        
        obs_dimensions = obs_space.shape[0]
        
        # policy network
        self.pi = Actor(obs_dimensions, act_space.n)
        
        # value network
        self.v = Critic(obs_dimensions)
        
        
    def step(self, obs):
        with torch.no_grad():
            pi = self.pi.get_distribution(obs)
            action = pi.sample()
            logp_a = pi.log_prob(action)
            val = self.v(obs)

        return action.numpy(), val.numpy(), logp_a.numpy()
    
    def get_action(self, obs):
        return self.step(obs)[0]
        

In [50]:
# Hyperparameter definitions
num_workers = 1
pi_lr = 3e-4
v_lr = 1e-3
epochs = 50 # 50
steps_per_epoch = 4000 // num_workers
max_ep_len = 1000
train_pi_iters = 80
train_v_iters = 80
gamma = 0.99
lam = 0.97
clip_ratio_offset = 0.2

In [51]:
def discount_cumsum(x, discount):
    return signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [72]:
def ppo():
    env = gym.make('LunarLander-v2')
    obs_space = env.observation_space
    act_space = env.action_space

    actor_critic = ActorCritic(obs_space, act_space)
    
    obs_buf  = np.zeros((steps_per_epoch, obs_space.shape[0]), dtype=np.float32)
    act_buf  = np.zeros(steps_per_epoch, dtype=np.float32)
    adv_buf  = np.zeros(steps_per_epoch, dtype=np.float32)
    rew_buf  = np.zeros(steps_per_epoch, dtype=np.float32)
    rew_boot_buf  = np.zeros(steps_per_epoch, dtype=np.float32)
    val_buf  = np.zeros(steps_per_epoch, dtype=np.float32)
    logp_buf = np.zeros(steps_per_epoch, dtype=np.float32)
    
    pi_optim = Adam(actor_critic.pi.parameters(), lr=pi_lr)
    v_optim = Adam(actor_critic.v.parameters(), lr=v_lr)
    
    def ppo_pi_loss(data):
        obs, action, advantage, old_logp = data['obs'], data['act'], data['adv'], data['logp']
        
        pi, logp = actor_critic.pi(obs, action)
        pi_ratio = torch.exp(logp - old_logp)
        clipped_adv = torch.clamp(pi_ratio, 1 - clip_ratio_offset, 1 + clip_ratio_offset) * advantage
        pi_loss = -(torch.min(pi_ratio * advantage, clipped_adv)).mean()
        return pi_loss
    
    def ppo_v_loss(data):
        obs, adj_rewards = data['obs'], data['rew']
        return (actor_critic.v(obs) - adj_rewards).pow(2).mean()
    
    def ppo_update():
        data = {k: torch.as_tensor(v, dtype=torch.float32)
                for k, v in dict(obs=obs_buf, act=act_buf, rew=rew_boot_buf, adv=adv_buf, logp=logp_buf).items()}
        
        for i in range(train_pi_iters):
            pi_optim.zero_grad()
            pi_loss = ppo_pi_loss(data)
            pi_loss.backward()
            pi_optim.step()
            
        for i in range(train_v_iters):
            v_optim.zero_grad()
            v_loss = ppo_v_loss(data)
            v_loss.backward()
            v_optim.step()

    obs, ep_reward, ep_len = env.reset(), 0, 0
    ep_reward_history = np.zeros(epochs, dtype=np.float32)
    for ep in range(epochs):
        for t in range(steps_per_epoch):
            action, val, logp_a = actor_critic.step(torch.as_tensor(obs, dtype=torch.float32))
            
            new_obs, reward, done, _ = env.step(action)
            ep_reward += reward
            ep_len += 1
            
            # buf.store(obs, action, reward, val, logp_a)
            obs_buf[t] = obs
            act_buf[t] = action
            rew_buf[t] = reward
            val_buf[t] = val
            logp_buf[t] = logp_a
            
            obs = new_obs
            
            timeout = ep_len == max_ep_len
            terminal = done or timeout
            epoch_ended = t==steps_per_epoch-1

            if terminal or epoch_ended:
                if timeout or epoch_ended:
                    _, bootstrap_v, _ = actor_critic.step(torch.as_tensor(obs, dtype=torch.float32))
                else:
                    bootstrap_v = 0

                rews_aug = np.append(rew_buf[:], bootstrap_v)
                vals_aug = np.append(val_buf[:], bootstrap_v)
                
                # GAE-Lambda advantage calculation
                gae_deltas = rews_aug[:-1] + gamma * vals_aug[1:] - vals_aug[:-1]
                adv_buf[:] = discount_cumsum(gae_deltas, gamma * lam)
        
                # Computes rewards-to-go, to be targets for the value function
                rew_boot_buf[:] = discount_cumsum(rews_aug, gamma)[:-1]

                ep_reward_history[ep] = max(ep_reward_history[ep], ep_reward)
                obs, ep_reward, done = env.reset(), 0, 0
        
        ppo_update()
        print(f"epoch: {ep + 1}, max reward: {ep_reward_history[ep]:.3f}")

In [73]:
ppo()

epoch: 1, max reward: 10.138
epoch: 2, max reward: 0.000
epoch: 3, max reward: 0.000
epoch: 4, max reward: 0.000
epoch: 5, max reward: 0.655
epoch: 6, max reward: 60.277
epoch: 7, max reward: 0.000
epoch: 8, max reward: 0.000
epoch: 9, max reward: 23.299
epoch: 10, max reward: 25.900
epoch: 11, max reward: 111.352
epoch: 12, max reward: 3.798
epoch: 13, max reward: 0.907
epoch: 14, max reward: 34.417
epoch: 15, max reward: 94.119
epoch: 16, max reward: 64.678
epoch: 17, max reward: 4.103
epoch: 18, max reward: 25.965
epoch: 19, max reward: 41.973
epoch: 20, max reward: 46.301
epoch: 21, max reward: 27.627
epoch: 22, max reward: 120.285
epoch: 23, max reward: 139.240
epoch: 24, max reward: 82.115
epoch: 25, max reward: 74.720
epoch: 26, max reward: 0.000
epoch: 27, max reward: 45.128
epoch: 28, max reward: 55.603
epoch: 29, max reward: 60.568
epoch: 30, max reward: 96.719
epoch: 31, max reward: 75.391
epoch: 32, max reward: 126.944
epoch: 33, max reward: 47.701
epoch: 34, max reward: 10